In [10]:
# import dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
# import urllib.parse
import requests
import pymongo
from pymongo import MongoClient
from selenium import webdriver
from splinter import Browser

In [52]:
# scrape news site for title and paragraph text, save off to variables
news_url = "https://mars.nasa.gov/news/"
driver = webdriver.Firefox()
driver.get(news_url)
html = driver.page_source
news_soup = bs(html, "html.parser")

news_title = (news_soup.find("div", class_="list_text")).find("a").text
print(news_title)

news_paragraph = news_soup.find("div", class_="article_teaser_body").text
print(news_paragraph)

#close firefox
driver.close()

The MarCO Mission Comes to an End
The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


In [54]:
# find featured Mars image from JPL, save full-size image url
# note: this url doesn't seem to pull up a Mars image any longer
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
base_url = "https://www.jpl.nasa.gov"
driver = webdriver.Firefox()
driver.get(jpl_url)
html = driver.page_source
img_soup = bs(html, "html.parser")

#build url from sytle tag
image_url  = img_soup.find("article")["style"].replace('background-image: url(','').replace(');', '')[1:-1]
featured_image_url = base_url + image_url
print(featured_image_url)
driver.close()

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16153-1920x1200.jpg


In [50]:
# scrape latest mars weather from twitter
mars_wx_url = "https://twitter.com/MarsWxReport"
r = requests.get(mars_wx_url)
html = r.text
wx_soup = bs(html, "html.parser")

mars_weather = wx_soup.find_all("div", class_="js-tweet-text-container")[0].text
print(mars_weather)


InSight sol 450 (2020-03-02) low -93.5ºC (-136.4ºF) high -10.4ºC (13.3ºF)
winds from the SSW at 5.5 m/s (12.4 mph) gusting to 20.6 m/s (46.1 mph)
pressure at 6.30 hPapic.twitter.com/82lzRqibcC



In [67]:
# scrape Mars facts website, display as html table str using pandas
mars_facts_url = 'https://space-facts.com/mars/'
mars_df = pd.read_html(mars_facts_url)[0]
mars_facts_html = mars_df.to_html('mars_facts.html')